In [346]:
# Code borrowed from https://github.com/anisfeld/MachineLearning/blob/master/Diagnostic/ML%20HW0-1.ipynb
import numpy as np
import pandas as pd
import requests
from urllib.parse import quote_plus, urlencode

# Socrata API interface functions

In [347]:
def encode_query(dict, base_url):
    '''
    Take dictionary with SQL statements parts and combine them into a web-safe string
    
    e.g.
    dict = {"SELECT": ["community_area",
                    "latitude",
                    "longitude",
                    "date"], 
            "WHERE": "iucr in ('2024','2025')",
            "LIMIT": "10"}
            
    returns: 'https://data.cityofchicago.org/resource/6zsd-86xi.json?$query=SELECT+community_area%2C+
              latitude%2C+longitude%2C+date+WHERE+iucr+in+%28%272024%27%2C%272025%27%29+LIMIT+10+'
    '''
    encoded = ""
    for key, val in dict.items():
        if isinstance(val, list):
            try:
                val = ", ".join(val)
            except:
                return "Lists must contain strings only"
        try:
            encoded += key + " " + val + " "
        except:
            return "Dictionary values must be strings or list of strings"
    return base_url + quote_plus(encoded)


def call_socrata_api(query_dict, base_url, endpoint=None, verbose=False):
    '''
    Inputs: endpoint (string) (e.g.: "yama-9had.json?", optional if base_url doesn't include one) 
            query_dict (dictionary) (SoQL statements divided into constituent parts)
    '''
    
    # Ensure all data is downloaded in one API call (requires endpoint version 2.1)
    count_query = base_url + "SELECT%20count(*)"
    r = requests.get(count_query)
    query_dict["LIMIT"] = r.json()[0]["count"]
    
    query_url = encode_query(query_dict, base_url)

    if verbose:
        print("query url: ", query_url)
    
    return pd.read_json(query_url)


def build_query_dict(code_data, base_query, code_col="IUCR", soql_name="iucr", group_col="Type", fn = chicago_codes):
    '''
    Put information about crime codes into usable format.
    
    code_data (pd.Dataframe) with two columns
    
    return: dict of form:
    {'Aggravated assault/battery': {'SELECT': ['id',
                                               'date',
                                               'iucr',
                                               'latitude',
                                               'longitude',
                                               'ward',
                                               'community_area',
                                               'district'],
                                    'WHERE': "iucr in ('141A', '141B')"},
    ...}
    '''
    soql_dict = {}
    types = code_data.groupby([group_col])
    for key,val in types.groups.items():
        soql_dict[key] = base_query.copy()
        codes = fn(code_data.ix[val,code_col])
        where = soql_name + " in "  + str(tuple(codes))
        soql_dict[key]['WHERE'] = where
    return soql_dict


def chicago_codes(col, n=4):
    return [str(x) if len(str(x)) == n else '0' + str(x) for x in col]

def new_york_codes(col):
    # Add a place holder so single-tuple format doesn't break the API syntax
    return chicago_codes(col, 3) + ['9999']

def make_crime_data_frame(query_dict, base_url, verbose=True):
    '''
    Idiosyncratic command that collects crime data and cleans up pandas dataframe 
    '''
    df = pd.DataFrame()
    for key, query in query_dict.items():
        temp_df = call_socrata_api(query, base_url, verbose=verbose)
        temp_df[key] = 1
        df = pd.concat([df,temp_df], axis=0)
    return df


# Chicago

In [421]:
BASE_URL = "https://data.cityofchicago.org/resource/6zsd-86xi.json?$query="
#https://data.cityofchicago.org/resource/6zsd-86xi.json?iucr=%270110%27
        
# Import IUCR codes with names.
code_data = pd.read_excel('Crime definitions.xlsx')
code_data.fillna(0, inplace=True)

Violent = ['Aggravated assault/battery', 'Criminal sexual assault', 'Homicide', 'Robbery']
Property = ['Arson','Burglary', 'Motor vehicle theft','Larceny']


# Build mapping from name to IUCR code
base_query = {"SELECT": ["id",
                        "date",
                        "iucr",
                        "latitude",
                        "longitude",
                        "ward",
                        "community_area",
                        "district"]}

query_dict = build_query_dict(code_data, base_query)
crime_types = list(query_dict.keys())

# pull crime data from Socrata
crime_data = make_crime_data_frame(query_dict, BASE_URL)
crime_data = crime_data.reset_index(drop=True)

crime_data.ix[:,crime_types] = crime_data.ix[:,crime_types ].fillna(0)

crime_data["Violent"] = crime_data[Violent].sum(1)
crime_data["Property"] = crime_data[Property].sum(1)
crime_data["Index"] = crime_data[["Violent","Property"]].sum(1)

crime_data.iucr = chicago_codes(crime_data.iucr)

public = code_data[["Public"]]
public["IUCR"] = pd.DataFrame(chicago_codes(code_data["IUCR"]))

crime_data = pd.merge(crime_data,public, left_on="iucr", right_on="IUCR")
crime_data.drop(["IUCR"], axis=1, inplace=True)
crime_data.to_csv("Chicago_Crimes.csv", index=False)



/Users/arianisfeld/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


# New York

In [330]:
BASE_URL = "https://data.cityofnewyork.us/resource/9s4h-37hy.json?$query="

code_data = pd.read_excel('NYPD.xlsx')
code_data.fillna(0, inplace=True)
crime_types = list(query_dict.keys())

Violent = ['Aggravated assault/battery', 'Criminal sexual assault', 'Homicide', 'Robbery']
Property = ['Arson','Burglary', 'Motor vehicle theft','Larceny']


# Build mapping from name to IUCR code
base_query = {"SELECT": ["cmplnt_num",
                        "cmplnt_fr_dt",
                        "cmplnt_fr_tm",
                        "ky_cd",
                        "latitude",
                        "longitude"]}

query_dict = build_query_dict(code_data, base_query,  code_col="KY_CD", soql_name="ky_cd", group_col="Type", fn=new_york_codes)
crime_types = list(query_dict.keys())

# pull crime data from Socrata 
#crime_data = make_crime_data_frame(query_dict, BASE_URL)
#crime_data = crime_data.reset_index(drop=True)
crime_data.ix[:,crime_types] = crime_data.ix[:,crime_types ].fillna(0)

crime_data["Violent"] = crime_data[Violent].sum(1)
crime_data["Property"] = crime_data[Property].sum(1)
crime_data["Index"] = crime_data[["Violent","Property"]].sum(1)

public = code_data[["Public","KY_CD"]]

crime_data = pd.merge(crime_data,public, left_on="ky_cd", right_on="KY_CD")

crime_data['date'] = crime_data.cmplnt_fr_dt.str.replace("T00:00:00.000"," ") + crime_data.cmplnt_fr_tm


crime_data.drop(["KY_CD","cmplnt_fr_dt","cmplnt_fr_tm"], axis=1, inplace=True)
crime_data.rename(columns = {'cmplnt_num':'id'}, inplace = True)

crime_data.to_csv("NY_Crimes.csv", index=False)

In [333]:
crime_data.to_csv("NY_Crimes.csv", index=False)

In [343]:
h = pd.read_csv("Chicago_Crimes.csv")
g = pd.read_csv("NY_Crimes.csv")

/Users/arianisfeld/anaconda2/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [379]:
g.describe()

,Aggravated assault/battery,Arson,Burglary,Criminal sexual assault,Drug abuse,Homicide,Larceny,Manslaughter,Motor vehicle theft,Robbery,Weapons violation,id,ky_cd,latitude,longitude,Violent,Property,Index,Public
count,2.668946e+06,2.668946e+06,2.668946e+06,2.668946e+06,2.668946e+06,2.668946e+06,2.668946e+06,2.668946e+06,2.668946e+06,2.668946e+06,2.668946e+06,2.668946e+06,2.668946e+06,2.592718e+06,2.592718e+06,2.668946e+06,2.668946e+06,2.668946e+06,2.668946e+06
mean,7.679399e-02,5.530273e-03,7.761004e-02,5.705623e-03,1.390571e-01,1.839303e-03,5.216677e-01,4.870837e-05,4.096561e-02,8.028375e-02,5.049784e-02,5.499336e+08,2.068767e+02,4.073716e+01,-7.392794e+01,1.646227e-01,6.457736e-01,8.103963e-01,2.151692e-01
std,2.662643e-01,7.415990e-02,2.675570e-01,7.531979e-02,3.460062e-01,4.284764e-02,4.995304e-01,6.978969e-03,1.982106e-01,2.717320e-01,2.189699e-01,2.598426e+08,1.061878e+02,8.098989e-02,7.315690e-02,3.708397e-01,4.782783e-01,3.919875e-01,4.109398e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000007e+08,1.010000e+02,4.049877e+01,-7.425508e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.248393e+08,1.090000e+02,4.067657e+01,-7.397743e+01,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00
50%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.497789e+08,1.180000e+02,4.073533e+01,-7.393223e+01,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00
75%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.751390e+08,3.410000e+02,4.081035e+01,-7.388725e+01,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00
max,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,9.999999e+08,5.770000e+02,4.091272e+01,-7.370032e+01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


### Scrap code


In [ ]:
def get_com_areas(df, right_on="area_numbe"):
    '''
    downloads community areas information from data portal and appends to data frame
    '''
    try:
        community_areas=pd.read_json("https://data.cityofchicago.org/resource/igwz-8jzy.json")
    except:
        return "download failed"
    
    com_areas = community_areas.ix[:,["area_numbe","community"]]
    df = pd.merge(df, com_areas, left_on="community_area", right_on=right_on)
    return df

g = code_dict.groupby(['Type'])
gr = g.groups['Aggravated assault/battery']
str(tuple([str(x) for x in code_dict.ix[gr,"IUCR"]]))
g.groups